# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f80b0f43d90>
├── 'a' --> tensor([[ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841]])
└── 'x' --> <FastTreeValue 0x7f80b0f43c70>
    └── 'c' --> tensor([[-1.1512,  0.5310, -1.6560,  0.4896],
                        [-0.7278, -0.3072,  0.9704,  1.4363],
                        [ 0.0175,  0.6630,  0.9286,  0.0258]])

In [4]:
t.a

tensor([[ 0.7690, -0.0159,  0.7301],
        [-0.2130,  0.8570,  0.7841]])

In [5]:
%timeit t.a

64.9 ns ± 0.0249 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f80b0f43d90>
├── 'a' --> tensor([[-0.9121, -1.2452, -0.9661],
│                   [-0.9420,  0.5393,  0.5141]])
└── 'x' --> <FastTreeValue 0x7f80b0f43c70>
    └── 'c' --> tensor([[-1.1512,  0.5310, -1.6560,  0.4896],
                        [-0.7278, -0.3072,  0.9704,  1.4363],
                        [ 0.0175,  0.6630,  0.9286,  0.0258]])

In [7]:
%timeit t.a = new_value

65.1 ns ± 0.0716 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7690, -0.0159,  0.7301],
               [-0.2130,  0.8570,  0.7841]]),
    x: Batch(
           c: tensor([[-1.1512,  0.5310, -1.6560,  0.4896],
                      [-0.7278, -0.3072,  0.9704,  1.4363],
                      [ 0.0175,  0.6630,  0.9286,  0.0258]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7690, -0.0159,  0.7301],
        [-0.2130,  0.8570,  0.7841]])

In [11]:
%timeit b.a

55.7 ns ± 0.0376 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4198, -3.2842,  1.4817],
               [-0.2325,  0.4614,  0.7535]]),
    x: Batch(
           c: tensor([[-1.1512,  0.5310, -1.6560,  0.4896],
                      [-0.7278, -0.3072,  0.9704,  1.4363],
                      [ 0.0175,  0.6630,  0.9286,  0.0258]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.148 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

848 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 37.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f80b0f43dc0>
├── 'a' --> tensor([[[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]],
│           
│                   [[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]],
│           
│                   [[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]],
│           
│                   [[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]],
│           
│                   [[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]],
│           
│                   [[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]],
│           
│                   [[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]],
│           
│                   [[ 0.7690, -0.0159,  0.7301],
│                    [-0.2130,  0.8570,  0.7841]]])
└── 'x' --> <FastTreeValue 0x7f80b0f433d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 41.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f80883b7970>
├── 'a' --> tensor([[ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841],
│                   [ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841],
│                   [ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841],
│                   [ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841],
│                   [ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841],
│                   [ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841],
│                   [ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841],
│                   [ 0.7690, -0.0159,  0.7301],
│                   [-0.2130,  0.8570,  0.7841]])
└── 'x' --> <FastTreeValue 0x7f80b0f22040>
    └── 'c' --> tensor([[-1.1512,  0.5310, -1.6560,  0.4896],
                        [-0.7278, -0.3072,  0.9704,  1.4363],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 29.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 63.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.1512,  0.5310, -1.6560,  0.4896],
                       [-0.7278, -0.3072,  0.9704,  1.4363],
                       [ 0.0175,  0.6630,  0.9286,  0.0258]],
              
                      [[-1.1512,  0.5310, -1.6560,  0.4896],
                       [-0.7278, -0.3072,  0.9704,  1.4363],
                       [ 0.0175,  0.6630,  0.9286,  0.0258]],
              
                      [[-1.1512,  0.5310, -1.6560,  0.4896],
                       [-0.7278, -0.3072,  0.9704,  1.4363],
                       [ 0.0175,  0.6630,  0.9286,  0.0258]],
              
                      [[-1.1512,  0.5310, -1.6560,  0.4896],
                       [-0.7278, -0.3072,  0.9704,  1.4363],
                       [ 0.0175,  0.6630,  0.9286,  0.0258]],
              
                      [[-1.1512,  0.5310, -1.6560,  0.4896],
                       [-0.7278, -0.3072,  0.9704,  1.4363],
                       [ 0.0175,  0.6630,  0.9286,  0.0258]],

In [26]:
%timeit Batch.stack(batches)

95.5 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.1512,  0.5310, -1.6560,  0.4896],
                      [-0.7278, -0.3072,  0.9704,  1.4363],
                      [ 0.0175,  0.6630,  0.9286,  0.0258],
                      [-1.1512,  0.5310, -1.6560,  0.4896],
                      [-0.7278, -0.3072,  0.9704,  1.4363],
                      [ 0.0175,  0.6630,  0.9286,  0.0258],
                      [-1.1512,  0.5310, -1.6560,  0.4896],
                      [-0.7278, -0.3072,  0.9704,  1.4363],
                      [ 0.0175,  0.6630,  0.9286,  0.0258],
                      [-1.1512,  0.5310, -1.6560,  0.4896],
                      [-0.7278, -0.3072,  0.9704,  1.4363],
                      [ 0.0175,  0.6630,  0.9286,  0.0258],
                      [-1.1512,  0.5310, -1.6560,  0.4896],
                      [-0.7278, -0.3072,  0.9704,  1.4363],
                      [ 0.0175,  0.6630,  0.9286,  0.0258],
                      [-1.1512,  0.5310, -1.6560,  0.4896],
                   

In [28]:
%timeit Batch.cat(batches)

163 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 2.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
